In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython .display import display
import os

In [2]:
#csv読み込み
df = pd.read_csv('./train.csv')
df2 = pd.read_csv('./test.csv')

In [40]:
#中身見る用
#AgeがNaN多すぎで他の列をもとに適当な値を入れたい
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
df.sort_values(by="Age", ascending=True).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
803,804,1,3,"Thomas, Master. Assad Alexander",male,0.42,0,1,2625,8.5167,NaN,C
755,756,1,2,"Hamalainen, Master. Viljo",male,0.67,1,1,250649,14.5000,NaN,S
644,645,1,3,"Baclini, Miss. Eugenie",female,0.75,2,1,2666,19.2583,NaN,C
469,470,1,3,"Baclini, Miss. Helene Barbara",female,0.75,2,1,2666,19.2583,NaN,C
78,79,1,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,248738,29.0000,NaN,S


In [39]:
print(df[df.isnull().any(axis=1)].shape)
df[df.isnull().any(axis=1)].head()

(689, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,30.726645,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S


In [10]:
df_1 = df.drop(['Cabin'], axis=1)
df_1[df_1.isnull().any(axis=1)].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,Q


In [16]:
df_2 = df[df['Age'] != 'NaN']
df_2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [37]:
for grp, data in df.groupby('Sex'):
    val2 = data['Age'].mean()
    df.loc[df['Sex']==grp,'Age'] = data['Age'].fillna(value=val2)

In [43]:
df.groupby(['Sex','Pclass'])['Fare'].median()

Sex     Pclass
female  1         82.66455
        2         22.00000
        3         12.47500
male    1         41.26250
        2         13.00000
        3          7.92500
Name: Fare, dtype: float64

In [49]:
#EmbarkedのnanにはSを入れる
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [51]:
#df2['Fare'] = df2['Fare'].fillna(df2['Fare'].median())
df.fillna(value={'Embarked': 'S'}, inplace=True)

In [52]:
#EmbarkedのnanにはSを入れる
df['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [53]:
#中身見る用
#AgeがNaN多すぎで他の列をもとに適当な値を入れたい
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [54]:
#加工
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#train_x = df.drop(['Name','PassengerId','Ticket','Cabin','Survived','Embarked','Age','Sex'], axis=1)
train_x = df.drop(['Name','PassengerId','Ticket','Cabin','Survived'], axis=1)
train_y = df['Survived']

In [55]:
#学習
forest = RandomForestClassifier(n_estimators=100, random_state= 41)
forest = forest.fit(train_x, train_y)
print("training set{:.3f}".format(forest.score(train_x, train_y)))

ValueError: could not convert string to float: 'male'

In [5]:
#testにNaNがあったためとりあえず中央値入れる
df2['Fare'] = df2['Fare'].fillna(df2['Fare'].median())
test = df2.drop(['Name','PassengerId','Ticket','Cabin','Embarked','Age','Sex'], axis=1)

yosoku = forest.predict(test)

In [6]:
yosoku.shape

(418,)

In [7]:
print(yosoku)

[0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0
 0 0 1 1 1 0 0 1 0 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0
 1 1 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1
 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0
 0 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 0 0
 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 1 0
 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1 1
 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0
 1 0 0 0 1 0 0 1 0 0 1]


In [8]:
#提出用のcsvを出力
PassengerId = np.array(df2['PassengerId']).astype(int)
dataframe = pd.DataFrame(yosoku,PassengerId, columns = ["Survived"])
dataframe.to_csv('teisyutu.csv', index_label = ['PassengerId'])